In [4]:
from llm_guard.vault import Vault

vault = Vault()

In [3]:
from llm_guard.input_scanners import Anonymize
from llm_guard.input_scanners.anonymize_helpers import 

scanner = Anonymize(vault, preamble="Insert before prompt", allowed_names=["John Doe"], hidden_names=["Test LLC"],
                    recognizer_conf=BERT_LARGE_NER_CONF, language="en")

2024-06-26 11:18:45 [debug    ] No entity types provided, using default default_entities=['CREDIT_CARD', 'CRYPTO', 'EMAIL_ADDRESS', 'IBAN_CODE', 'IP_ADDRESS', 'PERSON', 'PHONE_NUMBER', 'US_SSN', 'US_BANK_NUMBER', 'CREDIT_CARD_RE', 'UUID', 'EMAIL_ADDRESS_RE', 'US_SSN_RE']


Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2024-06-26 11:18:49 [debug    ] Initialized NER model          device=device(type='cuda', index=0) model=Model(path='dslim/bert-large-NER', subfolder='', revision='13e784dccceca07aee7a7aab4ad487c605975423', onnx_path='dslim/bert-large-NER', onnx_revision='13e784dccceca07aee7a7aab4ad487c605975423', onnx_subfolder='onnx', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='cuda', index=0), 'aggregation_strategy': 'simple'}, tokenizer_kwargs={'model_input_names': ['input_ids', 'attention_mask']})
2024-06-26 11:18:53 [debug    ] Loaded regex pattern           group_name=CREDIT_CARD_RE
2024-06-26 11:18:53 [debug    ] Loaded regex pattern           group_name=UUID
2024-06-26 11:18:53 [debug    ] Loaded regex pattern           group_name=EMAIL_ADDRESS_RE
2024-06-26 11:18:53 [debug    ] Loaded regex pattern           group_name=US_SSN_RE
2024-06-26 11:18:53 [debug    ] Loaded regex pattern           group_name=BTC_ADDRESS
2024-06-26 11:18:53 [debug  

In [3]:
prompt = "Kevin Mcarthur was a lonely man. He lived at 33 Drury Road, KT6 8SK. He likes swimming and fishing and recently had some trouble opening a bank account in Halifax. His email is kevmac@hotmail.com"

sanitized_prompt, is_valid, risk_score = scanner.scan(prompt)

2024-06-19 14:37:59 [debug    ] Ignoring entity                entity_group=LOCATION
2024-06-19 14:37:59 [debug    ] Ignoring entity                entity_group=LOCATION
2024-06-19 14:37:59 [debug    ] Ignoring entity                entity_group=LOCATION
2024-06-19 14:37:59 [debug    ] removing element type: EMAIL_ADDRESS_RE, start: 177, end: 195, score: 0.75 from results list due to conflict
2024-06-19 14:37:59 [warning  ] Found sensitive data in the prompt and replaced it merged_results=[type: PERSON, start: 0, end: 14, score: 0.949999988079071, type: EMAIL_ADDRESS, start: 177, end: 195, score: 1.0] risk_score=1.0


In [4]:
sanitized_prompt

'Insert before prompt[REDACTED_PERSON_1] was a lonely man. He lived at 33 Drury Road, KT6 8SK. He likes swimming and fishing and recently had some trouble opening a bank account in Halifax. His email is [REDACTED_EMAIL_ADDRESS_1]'

In [9]:
from llm_guard.input_scanners import Secrets

scanner = Secrets()
prompt = """

# syntax=docker/dockerfile:1

# NOTE: Building this image require's docker version >= 23.0.
#
# For reference:
# - https://docs.docker.com/build/dockerfile/frontend/#stable-channel

ARG BASE_IMAGE=ubuntu:22.04
ARG PYTHON_VERSION=3.11

FROM ${BASE_IMAGE} as dev-base
RUN apt-get update && DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        ccache \
        cmake \
        curl \
        git \
        libjpeg-dev \
        libpng-dev && \
    rm -rf /var/lib/apt/lists/*
RUN /usr/sbin/update-ccache-symlinks
RUN mkdir /opt/ccache && ccache --set-config=cache_dir=/opt/ccache
ENV PATH /opt/conda/bin:$PATH

FROM dev-base as conda
ARG PYTHON_VERSION=3.11
# Automatically set by buildx
ARG TARGETPLATFORM
# translating Docker's TARGETPLATFORM into miniconda arches
RUN case ${TARGETPLATFORM} in \
         "linux/arm64")  MINICONDA_ARCH=aarch64  ;; \
         *)              MINICONDA_ARCH=x86_64   ;; \
    esac && \
    curl -fsSL -v -o ~/miniconda.sh -O  "https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-${MINICONDA_ARCH}.sh"
COPY requirements.txt .
# Manually invoke bash on miniconda script per https://github.com/conda/conda/issues/10431
RUN chmod +x ~/miniconda.sh && \
    bash ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda install -y python=${PYTHON_VERSION} cmake conda-build pyyaml numpy ipython && \
    /opt/conda/bin/python -mpip install -r requirements.txt && \
    /opt/conda/bin/conda clean -ya

FROM dev-base as submodule-update
WORKDIR /opt/pytorch
COPY . .
RUN git submodule update --init --recursive

FROM conda as build
ARG CMAKE_VARS
WORKDIR /opt/pytorch
COPY --from=conda /opt/conda /opt/conda
COPY --from=submodule-update /opt/pytorch /opt/pytorch
RUN make triton
RUN --mount=type=cache,target=/opt/ccache \
    export eval ${CMAKE_VARS} && \
    TORCH_CUDA_ARCH_LIST="7.0 7.2 7.5 8.0 8.6 8.7 8.9 9.0 9.0a" TORCH_NVCC_FLAGS="-Xfatbin -compress-all" \
    CMAKE_PREFIX_PATH="$(dirname $(which conda))/../" \
    python setup.py install

FROM conda as conda-installs
ARG PYTHON_VERSION=3.11
ARG CUDA_VERSION=12.1
ARG CUDA_CHANNEL=nvidia
ARG INSTALL_CHANNEL=pytorch-nightly
# Automatically set by buildx
RUN /opt/conda/bin/conda update -y -n base -c defaults conda
RUN /opt/conda/bin/conda install -c "${INSTALL_CHANNEL}" -y python=${PYTHON_VERSION}

ARG TARGETPLATFORM

# On arm64 we can only install wheel packages.
RUN case ${TARGETPLATFORM} in \
         "linux/arm64")  pip install --extra-index-url https://download.pytorch.org/whl/cpu/ torch torchvision torchaudio ;; \
         *)              /opt/conda/bin/conda install -c "${INSTALL_CHANNEL}" -c "${CUDA_CHANNEL}" -y "python=${PYTHON_VERSION}" pytorch torchvision torchaudio "pytorch-cuda=$(echo $CUDA_VERSION | cut -d'.' -f 1-2)"  ;; \
    esac && \
    /opt/conda/bin/conda clean -ya
RUN /opt/conda/bin/pip install torchelastic

FROM ${BASE_IMAGE} as official
ARG PYTORCH_VERSION
ARG TRITON_VERSION
ARG TARGETPLATFORM
ARG CUDA_VERSION
LABEL com.nvidia.volumes.needed="nvidia_driver"
RUN apt-get update && DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
        ca-certificates \
        libjpeg-dev \
        libpng-dev \
        && rm -rf /var/lib/apt/lists/* \
        api_key = sk-w1ii3kalkdakla2
COPY --from=conda-installs /opt/conda /opt/conda
RUN if test -n "${TRITON_VERSION}" -a "${TARGETPLATFORM}" != "linux/arm64"; then \
        DEBIAN_FRONTEND=noninteractive apt install -y --no-install-recommends gcc; \
        rm -rf /var/lib/apt/lists/*; \
    fi
ENV PATH /opt/conda/bin:$PATH
ENV NVIDIA_VISIBLE_DEVICES all
ENV NVIDIA_DRIVER_CAPABILITIES compute,utility
ENV LD_LIBRARY_PATH /usr/local/nvidia/lib:/usr/local/nvidia/lib64
ENV PATH /usr/local/nvidia/bin:/usr/local/cuda/bin:$PATH
ENV PYTORCH_VERSION ${PYTORCH_VERSION}
ENV GCP_TOKEN ya29.c.b0Ae-h-sBAAAAAAAAAAAnp9CAdaXQivDNqDXdpTsSpDy3FrCDc6ZoUEZrwbV-vhZRrd_YeHJ3A7WuJUcqb7O3vCBBP2aPTVVvXZ
WORKDIR /workspace

FROM official as dev
# Should override the already installed version from the official-image stage
COPY --from=build /opt/conda /opt/conda

password = 123456789a1234
"""
sanitized_prompt, is_valid, risk_score = scanner.scan(prompt)
print(sanitized_prompt)

2024-06-19 14:53:20 [warning  ] Detected secrets in prompt     secret_types=['Hex High Entropy String']


# syntax=docker/dockerfile:1

# NOTE: Building this image require's docker version >= 23.0.
#
# For reference:
# - https://docs.docker.com/build/dockerfile/frontend/#stable-channel

ARG BASE_IMAGE=ubuntu:22.04
ARG PYTHON_VERSION=3.11

FROM ${BASE_IMAGE} as dev-base
RUN apt-get update && DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends         build-essential         ca-certificates         ccache         cmake         curl         git         libjpeg-dev         libpng-dev &&     rm -rf /var/lib/apt/lists/*
RUN /usr/sbin/update-ccache-symlinks
RUN mkdir /opt/ccache && ccache --set-config=cache_dir=/opt/ccache
ENV PATH /opt/conda/bin:$PATH

FROM dev-base as conda
ARG PYTHON_VERSION=3.11
# Automatically set by buildx
ARG TARGETPLATFORM
# translating Docker's TARGETPLATFORM into miniconda arches
RUN case ${TARGETPLATFORM} in          "linux/arm64")  MINICONDA_